In [1]:
import json
import pickle
import os
from datetime import datetime
import random
import numpy as np
import re
from text_util import strip_accents

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [3]:
from nltk.tokenize import sent_tokenize
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

In [4]:
random.seed(13)
torch.manual_seed(13)
np.random.seed(13)
torch.backends.cudnn.benchmarks = False
torch.use_deterministic_algorithms(True)
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"
device = torch.device('cuda:0')
plm_names = ['bert-base-uncased', 'bert-large-uncased', 'roberta-base', 'roberta-large', 'xlnet-large-cased', 'facebook/bart-large']
plm_name = plm_names[2]
plm_dim = 1024 if 'large' in plm_name else 768
model_names = ['basiccontext', 'sanitycheck', 'nocontext', 'userbiocontext', 'wikicontext', 'parcontext', 'parcontext-partial']
model_num = model_names[2]
num_folds = 3

In [5]:
tpath = '/homes/rpujari/scratch0_ml/tweet_target_data/'

In [6]:
with open(tpath + 'annotated_data_dict.pkl', 'rb') as infile:
    data_dict = pickle.load(infile)
with open(tpath + 'target_task_data_examples_v7.json', 'r') as infile:
    data_examples = json.load(infile)
with open(tpath + 'target_task_split_v7.json', 'r') as infile:
    data_split = json.load(infile)
with open(tpath + 'us_politicians.pkl', 'rb') as infile:
    politician_dict = pickle.load(infile)

In [7]:
c = 0
for tlink in data_examples:
    c += len(data_examples[tlink])
print(c)

5890


In [8]:
handler_mapping = dict()
for p_id in politician_dict:
    for cg_year in politician_dict[p_id]:
        handler = politician_dict[p_id][cg_year]['twitter_account']
        fqn = politician_dict[p_id][cg_year]['first_name'] + ' ' + politician_dict[p_id][cg_year]['last_name']
        fqn = strip_accents(fqn)
        if handler not in handler_mapping:
            handler_mapping[handler] = fqn
            
# Load PAR Embeddings
par_ent_list = [l.strip() for l in open(tpath + 'PAR/political_actor_reps/entity_list.txt').readlines() if l.strip()]
print(len(par_ent_list))
part_embs = torch.load(tpath + 'PAR/political_actor_reps/learned_reps.pt')
print(part_embs.shape)

1069
torch.Size([1069, 512])


In [9]:
with open(tpath + 'author_name_embeddings/author_name_embeddings_' + plm_name.replace('/', '_') + '.pkl', 'rb') as infile:
    author_name_embs = pickle.load(infile)
with open(tpath + 'user_description_embeddings/user_description_embeddings_' + plm_name.replace('/', '_') + '.pkl', 'rb') as infile:
    user_description_embs = pickle.load(infile)
with open(tpath + 'target_name_embeddings/target_name_embeddings_7.0_' + plm_name.replace('/', '_') + '.pkl', 'rb') as infile:
    target_name_embs = pickle.load(infile)
with open(tpath + 'event_name_embeddings/event_name_embeddings_' + plm_name.replace('/', '_') + '.pkl', 'rb') as infile:
    event_name_embs = pickle.load(infile)

### Embedding Computation

In [10]:
fnames = sorted([f for f in os.listdir(tpath + 'wiki_pages_7.0/') if not (f.endswith('.parse') or f.endswith('.pkl'))])

all_sents = []
sent_id_dict = {}
rev_sent_id_dict = {}

i = 0
for fname in fnames:
    ftext = sent_tokenize(open(tpath + 'wiki_pages_7.0/' + fname).read().strip())
    rev_sent_id_dict[fname] = []
    for sent in ftext:
        urls = re.findall('https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+', sent)
        for url in urls:
            sent = sent.replace(url, '')
        all_sents.append(sent)
        sent_id_dict[i] = fname
        rev_sent_id_dict[fname].append(i)
        i += 1

data_tlinks = sorted(list(data_dict.keys()))
for tlink in data_tlinks:
    ttext = data_dict[tlink]['text']
    urls = re.findall('https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+', ttext)
    for url in urls:
        ttext = ttext.replace(url, '')
    all_sents.append(ttext)    
    sent_id_dict[i] = tlink
    rev_sent_id_dict[tlink] = [i]
    i += 1

print(len(all_sents))

129639


In [11]:
with open(tpath + 'target_sentiment_embeddings/target_sentiment_7.0_all_embs_' + plm_name.replace('/', '_') + '.pkl', 'rb') as infile:
    all_embs = pickle.load(infile)

In [12]:
all_embs = torch.from_numpy(all_embs).type(torch.FloatTensor)
print(all_embs.size())

torch.Size([129639, 768])


In [13]:
file_embs = {}
for fname in fnames:
    fsent_embs = [all_embs[i:i+1, :] for i in rev_sent_id_dict[fname]]
    file_emb = torch.sum(torch.cat(fsent_embs, dim=0), dim=0)
    file_embs[fname] = file_emb.numpy().reshape(1, -1)
    
for tlink in data_dict.keys():
    data_dict[tlink]['tweet_emb'] = all_embs[rev_sent_id_dict[tlink][0]].numpy().reshape(1, -1)
    
for tlink in data_dict:
    eg = data_dict[tlink]
    file_embs[eg['text']] = eg['tweet_emb']

### Data Utils

In [14]:
event_wiki = {
    'capitol': 'https://en.wikipedia.org/wiki/2021_United_States_Capitol_attack',
    'blm': 'https://en.wikipedia.org/wiki/George_Floyd_protests',
    'lasvegas': 'https://en.wikipedia.org/wiki/2017_Las_Vegas_shooting',
    'borderwall': 'https://en.wikipedia.org/wiki/Trump_wall',
    'kavanugh': 'https://en.wikipedia.org/wiki/Brett_Kavanaugh_Supreme_Court_nomination',
    'elpaso': 'https://en.wikipedia.org/wiki/2019_El_Paso_shooting'
}

rev_event_wiki = {}
for event in event_wiki:
    rev_event_wiki[event_wiki[event]] = event

### Data Tensor Generation

In [15]:
def get_data_arrays(data_examples):
    data_x = []
    data_y = []

    data_tlinks = sorted(list(data_examples.keys()))
    wiki_count = 0

    for tlink in data_tlinks:
        #read example from data_dict
        eg = data_dict[tlink]
        sel_eg = True

        #nocontext mode means embeddings come from PLM embedding of the name strings
        if model_num == 'nocontext' or model_num == 'basiccontext':
            author_emb = torch.from_numpy(author_name_embs[tlink].reshape(-1)).type(torch.FloatTensor) #model-0
            event_emb = torch.from_numpy(event_name_embs[eg['event']].reshape(-1)).type(torch.FloatTensor) #model-0

        #userbiocontext -> author embeddings from author's twitter bio
        elif model_num == 'userbiocontext':
            author_emb = torch.from_numpy(user_description_embs[tlink].reshape(-1)).type(torch.FloatTensor) #model-1
            event_emb = torch.from_numpy(event_name_embs[eg['event']].reshape(-1)).type(torch.FloatTensor) #model-1

        #wikicontext -> author and event embeddings come from their respective wikipedia pages
        elif model_num == 'wikicontext':
            author_emb = torch.from_numpy(file_embs[eg['author']].reshape(-1)).type(torch.FloatTensor) #model-2
            event_emb = torch.from_numpy(file_embs[rev_event_wiki[eg['event']]].reshape(-1)).type(torch.FloatTensor) #model-2

        #parcontext-partial -> author embeddings come from PAR model, event embeddings from wikipedia pages
        #parcontext -> use wiki page embedding for missing authors
        elif model_num == 'parcontext' or model_num == 'parcontext-partial':
            if eg['author'][1:] in handler_mapping:
                real_name = handler_mapping[eg['author'][1:]]
                if real_name.lower() in par_ent_list:
                    author_emb = part_embs[par_ent_list.index(real_name.lower()), :]
                else:
                    if model_num == 'parcontext-partial':
                        sel_eg = False
                    author_emb = torch.from_numpy(file_embs[eg['author']].reshape(-1)[:512]).type(torch.FloatTensor) #model-2
                    wiki_count += 1
            else:
                if model_num == 'parcontext-partial':
                    sel_eg = False
                author_emb = torch.from_numpy(file_embs[eg['author']].reshape(-1)[:512]).type(torch.FloatTensor) #model-2
                wiki_count += 1
            event_emb = torch.from_numpy(file_embs[rev_event_wiki[eg['event']]].reshape(-1)).type(torch.FloatTensor) #model-2

        tweet_emb = torch.from_numpy(eg['tweet_emb'].reshape(-1)).type(torch.FloatTensor)

        for eg in data_examples[tlink]:
            if model_num == 'nocontext' or model_num == 'userbiocontext' or model_num == 'sanitycheck' or model_num == 'basiccontext': 
                ann_emb = torch.from_numpy(target_name_embs[eg['choice']].reshape(-1)).type(torch.FloatTensor) #model-0/model-1
            elif model_num == 'wikicontext' or model_num == 'parcontext' or model_num == 'parcontext-partial' :
                ann_emb = torch.from_numpy(file_embs[eg['choice']].reshape(-1)).type(torch.FloatTensor) #model-2
            if sel_eg:
                if model_num == 'sanitycheck':
                    data_x.append((ann_emb))
                elif model_num == 'basiccontext':
                    data_x.append((tweet_emb, ann_emb))
                else:
                    data_x.append((author_emb, event_emb, tweet_emb, ann_emb))
                data_y.append(torch.tensor(np.array([int(eg['label'])])))

    print(len(data_x))
    if model_num == 'parcontext' or model_num == 'parcontext-partial':
        print(wiki_count)
        
    return data_x, data_y

In [16]:
# data_x, data_y = get_data_arrays(data_examples)

# data_inds = list(range(len(data_x)))
# random.shuffle(data_inds)a
# # print(data_inds)
# data_x = [data_x[i] for i in data_inds]
# data_y = [data_y[i] for i in data_inds]

In [17]:
# folds = []

# for i in range(num_folds):
#     folds.append(([], []))
    
# for x, y in zip(data_x, data_y):
#     fnum = int(np.floor(random.random() * num_folds))
#     if model_num == 'sanitycheck':
#         folds[fnum][0].append(x.unsqueeze(0))
#     else:
#         folds[fnum][0].append(torch.cat(list(x), dim=0).unsqueeze(0))
#     folds[fnum][1].append(y)
    
# tensor_data = []
# for i in range(num_folds):
#     fold_inds = list(range(len(folds[i][0])))
#     random.shuffle(fold_inds)
#     # print(fold_inds, '\n')
#     fold_x = [folds[i][0][j] for j in fold_inds]
#     fold_y = [folds[i][1][j] for j in fold_inds]
#     # print(fold_x[0].shape)
#     tensor_data.append((torch.cat(fold_x, dim=0), torch.cat(fold_y, dim=0)))

In [18]:
# print(tensor_data[0][0].size())

In [19]:
tr_data_x, tr_data_y = get_data_arrays(data_split['train'])
de_data_x, de_data_y = get_data_arrays(data_split['dev'])
te_data_x, te_data_y = get_data_arrays(data_split['test'])

4370
511
1009


In [20]:
data_inds = list(range(len(tr_data_x)))
random.shuffle(data_inds)

tr_data_x = [tr_data_x[i] for i in data_inds]
tr_data_y = [tr_data_y[i] for i in data_inds]

#train_tensor creation
tr_data = ([], [])
for x, y in zip(tr_data_x, tr_data_y):
    if model_num == 'sanitycheck':
        tr_data[0].append(x.unsqueeze(0))
    else:
        tr_data[0].append(torch.cat(list(x), dim=0).unsqueeze(0))
    tr_data[1].append(y)

tr_tensor = (torch.cat(tr_data[0], dim=0), torch.cat(tr_data[1], dim=0))

#dev_tensor creation
de_data = ([], [])
for x, y in zip(de_data_x, de_data_y):
    if model_num == 'sanitycheck':
        de_data[0].append(x.unsqueeze(0))
    else:
        de_data[0].append(torch.cat(list(x), dim=0).unsqueeze(0))
    de_data[1].append(y)

de_tensor = (torch.cat(de_data[0], dim=0), torch.cat(de_data[1], dim=0))

#test_tensor creation
te_data = ([], [])
for x, y in zip(te_data_x, te_data_y):
    if model_num == 'sanitycheck':
        te_data[0].append(x.unsqueeze(0))
    else:
        te_data[0].append(torch.cat(list(x), dim=0).unsqueeze(0))
    te_data[1].append(y)

te_tensor = (torch.cat(te_data[0], dim=0), torch.cat(te_data[1], dim=0))

In [21]:
split_tensor = {
    'train': tr_tensor,
    'dev': de_tensor,
    'test': te_tensor
}

### Model

In [22]:
class FeedForward(nn.Module):
    
    def __init__(self, sizes, use_cuda, lr, momentum, weight_decay, loss_weight):
        super(FeedForward, self).__init__()
        self.layers = []
        i = 0
        for s1, s2 in zip(sizes[:-1], sizes[1:]):
            self.layers.append(nn.Linear(s1, s2))
            self.register_parameter('weight-layer-' + str(i), self.layers[-1].weight)
            self.register_parameter('bias-layer-' + str(i), self.layers[-1].bias)
            nn.init.xavier_uniform_(self.layers[-1].weight)
            i += 1
        self.loss_fn = nn.CrossEntropyLoss(weight=loss_weight)
        self.nl = nn.Tanh()
        params = [p for p in self.parameters() if p.requires_grad]
        self.optimizer = optim.Adam(params, lr=lr)
        self.use_cuda = use_cuda
        
    def forward(self, x):
        for layer in self.layers:
            x = self.nl(layer(x))
        out = F.softmax(x, dim=1)
        return out
        
    def evaluate(self, data):
        self.eval()
        batch_outs = []
        batch_preds = []
        batch_ys = []
        for batch_x, batch_y in data:
            if self.use_cuda:
                batch_x = batch_x.cuda(device)
                batch_y = batch_y.cuda(device)
            batch_out = self.forward(batch_x)
            batch_pred = torch.argmax(batch_out, dim=1)
            batch_outs.append(batch_out)
            batch_preds.append(batch_pred)
            batch_ys.append(batch_y)
        pred_y = torch.cat(batch_preds, dim=0)
        data_y = torch.cat(batch_ys, dim=0)
        pred_out = torch.cat(batch_outs, dim=0)
        acc = sum((pred_y == data_y).float()) / data_y.size(0)
        f1_mi = f1_score(data_y.cpu().data, pred_y.cpu().data, average='micro')
        f1_ma = f1_score(data_y.cpu().data, pred_y.cpu().data, average='macro')
        con_mat = confusion_matrix(data_y.cpu().data, pred_y.cpu().data)
        val_loss = self.loss_fn(pred_out, data_y)
        return float(acc.cpu().numpy()), float(val_loss.cpu().detach().numpy()), (f1_mi, f1_ma, [[int(x) for x in list(con_mat[0])], [int(x) for x in list(con_mat[1])]]), pred_y
        
    def train_model(self, train, val, num_epochs=25, save_path='./model.pkl', dev_benchmark=None):
        self.train()
        if dev_benchmark:
            max_val = dev_benchmark[2][1]
        else:
            max_val = -1
        for i in range(num_epochs):
            for batch_x, batch_y in train:
                if self.use_cuda:
                    batch_x = batch_x.cuda(device)
                    batch_y = batch_y.cuda(device)
                ids = list(range(batch_x.size(0)))
                # random.shuffle(ids)
                batch_x = batch_x[ids, :]
                batch_y = batch_y[ids]
                batch_out = self.forward(batch_x)
                loss = self.loss_fn(batch_out, batch_y)
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()
            val_acc, val_loss, (f1_mi, f1_ma, cm), _ = self.evaluate(val)
            if f1_ma > max_val:
                max_val = f1_ma
                torch.save(self.state_dict(), save_path)

In [23]:
def batchify_data(data, bsz=8):
    dlen = data[0].size(0)
    beg = 0
    end = bsz
    data_batches = []
    while beg < dlen:
        data_batches.append((data[0][beg:end, :], data[1][beg:end]))
        beg += bsz
        end += bsz
    return data_batches

In [24]:
def k_fold_cv(data, args, k=10, num_epochs=25):
    performance = []
    for i in range(k):
        test_data = batchify_data(data[i])
        val_data = batchify_data(data[(i + 1) % k])
        train_x = []
        train_y = []
        
        for j in range(k):
            if j != i and j != (i + 1) % k:
                train_x.append(data[j][0])
                train_y.append(data[j][1])
        
        train_X = torch.cat(train_x, dim=0)
        train_Y = torch.cat(train_y, dim=0)
        train_data = batchify_data((train_X, train_Y))
        
        label_loss_weight = train_Y.size(0) / torch.bincount(train_Y)
        label_loss_weight = label_loss_weight / torch.sum(label_loss_weight)
        args.loss_weight = label_loss_weight
        
        print(args.loss_weight)
        model = FeedForward(args.sizes, args.use_cuda, args.lr, args.momentum, args.weight_decay, args.loss_weight)
        model.cuda(device)
        model.train_model(train_data, val_data, num_epochs, save_path='/homes/rpujari/scratch0_ml/trained_models/trained_params/baseline-model-7.0-' + str(model_num) + '-' + plm_name.replace('/', '_') + '-fold-' + str(i) + '.pkl')
        model.load_state_dict(torch.load('/homes/rpujari/scratch0_ml/trained_models/trained_params/baseline-model-7.0-' + str(model_num) + '-' + plm_name.replace('/', '_') + '-fold-' + str(i)  + '.pkl'))
        performance.append(model.evaluate(test_data))
    return performance

In [25]:
def train_test_experiments(data, args, num_epochs=25):
    performance = {}
    train_fold = data['train']
    val_fold = data['dev']
    test_fold = data['test']
    
    train_data = batchify_data(train_fold)
    val_data = batchify_data(val_fold)
    test_data = batchify_data(test_fold)
    
    label_loss_weight = train_fold[1].size(0) / torch.bincount(train_fold[1])
    label_loss_weight = label_loss_weight / torch.sum(label_loss_weight)
    args.loss_weight = label_loss_weight
    print(args.loss_weight)
    
    model = FeedForward(args.sizes, args.use_cuda, args.lr, args.momentum, args.weight_decay, args.loss_weight)
    model.to(device)
    model.train_model(train_data, val_data, num_epochs, save_path='/homes/rpujari/scratch0_ml/trained_models/trained_params/baseline-split-7.0-' + str(model_num) + '-' + plm_name.replace('/', '_') + '.pkl')
    model.load_state_dict(torch.load('/homes/rpujari/scratch0_ml/trained_models/trained_params/baseline-split-7.0-' + str(model_num) + '-' + plm_name.replace('/', '_') + '.pkl'))
    
    performance['train'] = model.evaluate(train_data)
    performance['dev'] = model.evaluate(val_data)
    performance['test'] = model.evaluate(test_data)
    return performance

In [26]:
def train_test_inference(data, args):
    performance = {}
    train_fold = data['train']
    val_fold = data['dev']
    test_fold = data['test']
    
    train_data = batchify_data(train_fold)
    val_data = batchify_data(val_fold)
    test_data = batchify_data(test_fold)
    
    label_loss_weight = train_fold[1].size(0) / torch.bincount(train_fold[1])
    label_loss_weight = label_loss_weight / torch.sum(label_loss_weight)
    args.loss_weight = label_loss_weight
    print(args.loss_weight)
    
    model = FeedForward(args.sizes, args.use_cuda, args.lr, args.momentum, args.weight_decay, args.loss_weight)
    model.to(device)
    model.load_state_dict(torch.load('/homes/rpujari/scratch0_ml/trained_models/trained_params/baseline-split-7.0-' + str(model_num) + '-' + plm_name.replace('/', '_') + '.pkl', map_location=device))
    
    performance['train'] = model.evaluate(train_data)
    performance['dev'] = model.evaluate(val_data)
    performance['test'] = model.evaluate(test_data)
    return performance

### Experiments

In [27]:
class Args():
    def __init__(self):
        if model_num == 'parcontext' or model_num == 'parcontext-partial':
            self.sizes = [512 + 3 * plm_dim, 1000, 2]
        elif model_num == 'sanitycheck':
            self.sizes = [plm_dim, 1000, 2]
        elif model_num == 'basiccontext':
            self.sizes = [2 * plm_dim, 1000, 2]
        else:
            self.sizes = [4 * plm_dim, 1000, 2]
        self.use_cuda = True
        self.lr = 1e-5
        self.momentum = 0
        self.weight_decay = 0
        self.loss_weight = torch.tensor(np.array([0.5, 0.5]))
        
args1 = Args()

In [28]:
t1 = datetime.now()
# res = k_fold_cv(tensor_data, args1, k=num_folds, num_epochs=75)
# res = train_test_experiments(split_tensor, args1, num_epochs=100)
res = train_test_inference(split_tensor, args1)
t2 = datetime.now()
print(t2 - t1)

tensor([0.5515, 0.4485])
0:00:03.115977


In [29]:
# result_log = '../result_logs/baseline-split-7.0-' + str(model_num) + '-' +  plm_name.replace('/', '-') + '.log'
# result_log = open(result_log, 'w')

# f1_mas = [res[r1][2][1] for r1 in res]
# for key in res:
#     r1 = res[key]
#     cmat = np.array(r1[2][2])
#     prec = cmat[1, 1] / (cmat[1, 1] + cmat[0, 1])
#     rec = cmat[1, 1] / (cmat[1, 1] + cmat[1, 0])
#     acc = (cmat[1, 1] + cmat[0, 0]) / (cmat[0, 0] + cmat[0, 1] + cmat[1, 0] + cmat[1, 1])
#     result_log.write(key + '\n')
#     result_log.write('Precision: ' + str(round(prec * 100, 2))  + '\n')
#     result_log.write('Recall: ' + str(round(rec * 100, 2)) + '\n')
#     result_log.write('F1: ' + str(round(r1[2][1] * 100, 2)) + '\n')
#     result_log.write('Accuracy: ' + str(round(acc * 100, 2)) + '\n\n')
#     result_log.write('Confusion Matrix:\n')
#     result_log.write(str(r1[2][2]))
#     result_log.write('\n\n')
    
# result_log.close()

In [30]:
import copy
res_test = res['test'][-1]
i = 0
out_file = 'baseline-split-7.0-' + plm_name.replace('/', '_') + '-' + str(model_num) + '-hr-output.json'
predidction_human = {}
for tlink in data_split['test']:
    predidction_human[tlink] = []
    for ann in data_split['test'][tlink]:
        hr_ann = copy.deepcopy(ann)
        hr_ann['prediction'] = int(res_test[i].data)
        predidction_human[tlink].append(hr_ann)
        i += 1
        
json.dump(predidction_human, open(tpath + 'human_readable_outputs/' + out_file, 'w'))

In [31]:
# result_log = '../result_logs/baseline-model-6.0-' + str(model_num) + '-' +  plm_name.replace('/', '-') + '.log'
# result_log = open(result_log, 'w')

# f1_mas = [r1[2][1] for r1 in res]
# for i, r1 in enumerate(res):
#     result_log.write('Fold-' + str(i) + ' Mac-F1:')
#     result_log.write(str(round(r1[2][1] * 100, 2)))
#     result_log.write('\n\n')
#     result_log.write('Fold-' + str(i) + ' Confusion Matrix:\n')
#     result_log.write(str(r1[2][2]))
#     result_log.write('\n\n')
# result_log.write('Macro-F1:')
# result_log.write(str(round(np.mean(f1_mas) * 100, 2)))
# result_log.write('\n\n')

# precs = []
# accs = []
# recs = []
# for r1 in res:
#     cmat = np.array(r1[2][2])
#     prec = cmat[1, 1] / (cmat[1, 1] + cmat[0, 1])
#     rec = cmat[1, 1] / (cmat[1, 1] + cmat[1, 0])
#     acc = (cmat[1, 1] + cmat[0, 0]) / (cmat[0, 0] + cmat[0, 1] + cmat[1, 0] + cmat[1, 1])
#     precs.append(prec)
#     accs.append(acc)
#     recs.append(rec)
# result_log.write('Precision:')
# result_log.write(str(round(np.mean(precs) * 100, 2)))
# result_log.write('\n\n')
# result_log.write('Recall:')
# result_log.write(str(round(np.mean(recs) * 100, 2)))
# result_log.write('\n\n')
# result_log.write('Accuracy:')
# result_log.write(str(round(np.mean(accs) * 100, 2)))
# result_log.write('\n\n')
# result_log.close()